In [1]:
import unittest
import os

# please restart the kernel if rerunning this for the paths to align
os.chdir('../')

import numpy as np
import pandas as pd

from mswh.comm.label_map import SwhLabels
from mswh.system.components import Converter, Storage
from mswh.system.source_and_sink import SourceAndSink
from mswh.system.models import System
from mswh.system.source_and_sink import SourceAndSink

from mswh.tools.unit_converters import UnitConv, Utility
from mswh.comm.sql import Sql

from mswh.tools.plots import Plot
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
sns.set(rc={'figure.figsize':(16,12)})
sns.set_style("whitegrid")

import datetime

import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

# label maps
c = SwhLabels().set_hous_labels()
s = SwhLabels().set_prod_labels()
r = SwhLabels().set_res_labels()

# pd.options.display.float_format = '{:,.3f}'.format


# Multiscale Solar Water Heating System Simulation Tool 

## Contents


* **Project input (minimum user requirment)**
* **Weather (CA climate zones available in the database)** 
* **Loads (example CA hourly loads available in the database)**
* **Performance paramters (defaults provided, user edits optional, such as providing insulation thickness for tanks)**
* **Component sizing (sizing rules provided, user edits optional, such as providing the actual length of the piping)**
* **System simulation examples with plots and summaries for all system configurations**

### System configurations
* Basecase gas tank water heater (WH)
* Active solar thermal system with a new tankless gas WH backup
* Active solar thermal system with a retrofit tankless gas WH backup


# Setup 
## Import modules 

## Project inputs
* The user in the default case only needs to specify household occupancy and a climate in CA

In [2]:
# Add a value to the list for each additional household in a community scale project

# Community scale household occupancies for 4 households
occ_com = [3, 3, 3, 3]

# Individual scale household occupancy
occ_ind = [3]

# indicate whether any occupants are at home during the day
at_home_com = ['n', 'n', 'n', 'n']
at_home_ind = ['n']

# Example climate zones in CA
cold_cz = '01'
hot_cz = '15'
val_cz = '03'

## Load in input database (DB)

In [3]:
weather_db_path = os.path.join(os.getcwd(), 'mswh/comm/mswh_system_input.db')

db = Sql(weather_db_path)

try:
    # read table names for all tables in a
    # {table name : sheet name} form
    inputs = db.tables2dict(close = True)
except:
    msg = 'Failed to read input tables from {}.'
    log.error(msg.format(inpath))

## Generate weather data
* Includes calculation of the incident irradiation with the default tilt and azimuth

In [4]:
source_and_sink = SourceAndSink(\
    input_dfs = inputs)

weather_cold = source_and_sink.irradiation_and_water_main(
    cold_cz, method='isotropic diffuse')

weather_hot = source_and_sink.irradiation_and_water_main(
    hot_cz, method='isotropic diffuse')

weather_val = source_and_sink.irradiation_and_water_main(
    val_cz, method='isotropic diffuse')

INFO:mswh.system.source_and_sink:The user did not provide a numpy random state object. Initiating one with a provided or default seed value = 123.


## Select hourly hot water draw profiles from DB

In [5]:
# Returns hourly load profiles in m3/h and the peak load in gal/h for each household in the project

rs = np.random.RandomState(123)

# peakload_com output shows the consumer ID, load ID and the peak hourly end-use load
# loads_com holds the loads itself, with the associated consumer ID and occupancy

loads_com, peakload_com = \
    SourceAndSink._make_example_loading_inputs(
    inputs, c, rs,
    occupancy = occ_com, at_home = at_home_com)

loads_indiv, peakload_indiv = \
    SourceAndSink._make_example_loading_inputs(
    inputs, c, rs,
    occupancy = occ_ind, at_home = at_home_ind)

peakload_com

,Consumer ID,Load ID,Peak End-Use Load [gal]
0,1,109,40.00
1,2,88,42.00
2,3,46,42.00
3,4,73,44.75


In [6]:
peakload_indiv

,Consumer ID,Load ID,Peak End-Use Load [gal]
0,1,46,42.0


## Set performance parameters

* Active solar thermal system

In [7]:
# Community scale system (includes distribution pump parameters)

sol_the_sys_params_com = pd.DataFrame(data = \
            [[s['the_sto'], s['f_upper_vol'], .5], \
             [s['the_sto'], s['ins_thi'], .085], \
             [s['the_sto'], s['spec_hea_con'], .04], \
             [s['the_sto'], s['t_tap_set'], 322.04], \
             [s['the_sto'], s['h_vs_r'], 6.], \
             [s['the_sto'], s['dt_appr'], 2.], \
             [s['the_sto'], s['t_max_tank'], 344.15], \
             [s['the_sto'], s['eta_coil'], .84], \
             [s['the_sto'], s['circ'], 0.], \
             [s['sol_col'], s['interc_hwb'], .753], \
             [s['sol_col'], s['slope_hwb'], -4.025],\
             [s['sol_pump'], s['eta_sol_pump'], .85], \
             [s['piping'], s['pipe_spec_hea_con'], .0175], \
             [s['piping'], s['pipe_ins_thick'], .008], \
             [s['piping'], s['flow_factor'], .8], \
             [s['piping'], s['dia_len_exp'], 0.43082708345352605], \
             [s['piping'], s['dia_len_sca'], 0.007911283766743384],\
             [s['piping'], s['discr_diam_m'], \
               '[0.0127, 0.01905, 0.0254, '\
               '0.03175, 0.0381, 0.0508, 0.0635, 0.0762, 0.1016]'],\
             [s['piping'], s['circ'], False], \
             [s['piping'], s['long_br_len_fr'], 1.], \
             [s['dist_pump'], s['eta_dist_pump'], .85]], \
            columns = \
             [s['comp'], s['param'], s['param_value']])

sol_the_sys_params_com

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


In [8]:
# Individual scale system, assuming a distribution pump is not needed similarly to a basecase gas tank WH

last_row_for_indiv = sol_the_sys_params_com.shape[0] - 1 - \
    sol_the_sys_params_com.loc[\
    sol_the_sys_params_com[s['comp']] == s['dist_pump'], :].\
    shape[0]

sol_the_sys_params_indiv = sol_the_sys_params_com.loc[:last_row_for_indiv,:]
sol_the_sys_params_indiv

sol_the_sys_params_indiv

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


* Basecase and backup gas tank water heater (WH)

In [9]:
gas_tank_wh_params = pd.DataFrame(data = \
    [[s['gas_tank'], s['tank_re'], .78, '-'], \
     [s['gas_tank'], s['ins_thi'], .03, 'm'], \
     [s['gas_tank'], s['spec_hea_con'], .081, 'W/mK'], \
     [s['gas_tank'], s['t_tap_set'], 322.04, 'K']], \
    columns = \
     [s['comp'], s['param'], s['param_value'], s['param_unit']])

gas_tank_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas tank WH,tank recovery efficiency,0.780,-
1,gas tank WH,insulation thickness,0.030,m
2,gas tank WH,specific heat conductivity,0.081,W/mK
3,gas tank WH,tap temperature setpoint,322.040,K


* Gas tankless WH

In [10]:
gas_tankless_wh_params = pd.DataFrame(data = [[s['gas_burn'], s['comb_eff'], .85, '-']], \
    columns = [s['comp'], s['param'], s['param_value'], s['param_unit']])
gas_tankless_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas burner,combustion efficiency,0.85,-


* All performance parameters

In [11]:
# this input table provides all available component performance parameters
inputs['component_performance']

,Component ID,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,1,interc hwb,0.753,-
1,1,slope hwb,-4.025,W/m2K
2,1,interc cd,0.75,-
3,1,a1 cd,-3.688,W/m2K
4,1,a2 cd,-0.0055,W/m2K2
5,4,PV efficiency,0.16,-
6,4,fraction of active PV area,1,-
7,4,reference irradiation,1000,W/m2
8,5,tank recovery efficiency,0.78,-
9,5,tap temperature setpoint,322.04,K


In [12]:
inputs['sys_3_components']

,Component ID,Component,Component Technology,Component Size Unit
0,1,solar collector,flat plate,m2
1,2,solar collector,tubular,m2
2,3,pv,monochrystalline,W
3,4,pv,polychrystalline,W
4,5,gas tank WH,conventional gas tank water heater,m3
5,6,electric resistance tank WH,conventional electric resistance tank water he...,m3
6,7,thermal storage tank,thermal storage tank with an in-tank coil,m3
7,8,gas burner,gas burner,W
8,9,electric resistance heater,in-tank or instantaneous,W
9,10,heat pump,in-tank,W


In [13]:
to_doc = inputs['component_performance'].merge(inputs['sys_3_components'], on = "Component ID", how = "left")

## Component sizing

Note that backup is sized to full load.

* Gas tank WH: DOE sizing rule based on peak hourly demand +-2 gal

In [14]:
# Basecase gas tank WH

# in m3
gas_tank_size_indiv = UnitConv(peakload_indiv.loc[0, c['max_load']]).m3_gal(unit_in = 'gal')
gas_tank_size_indiv

# Individual household, backup
# same as the basecase sizing

0.158987304

In [15]:
# Community scale, backup

# get peak loads for each household
peakload_com['gas_tank_size_com'] = peakload_com[\
    c['max_load']].apply(lambda x: UnitConv(x).m3_gal(unit_in = 'gal'))
peakload_com.index = peakload_com[c['id']]

peakload_com

,Consumer ID,Load ID,Peak End-Use Load [gal],gas_tank_size_com
Consumer ID,,,,
1,1,109,40.00,0.151416
2,2,88,42.00,0.158987
3,3,46,42.00,0.158987
4,4,73,44.75,0.169397


* Active solar thermal system: CSI Handbook sizing rules

In [16]:
# CSI demand estimate
def demand_estimate(occ):
    if occ == 1:
        return 20.
    if occ == 2:
        return 35.
    else:
        return 35.+10.*(occ-2.)
    
col_area_scaler = 1.2 #(CSI sizing rule: upper limit 1.25)
tank_vol_scaler = 1.3 #(CSI sizing rule: lower limit 1.25)

# Individual household (without a distribution pump)

# get demand estimate
demand_estimate_ind = demand_estimate(occ_ind[0])

# CSI sizing rules
col_area_ind_sqft = demand_estimate_ind * col_area_scaler
tank_vol_ind_gal = col_area_ind_sqft * tank_vol_scaler

col_area_ind = UnitConv(col_area_ind_sqft).sqft_m2()
tank_vol_ind = UnitConv(tank_vol_ind_gal).m3_gal(unit_in = 'gal')

# Community scale (with a distribution pump)

# get demand estimate
demand_estimate_com = 0.
for inx, row in loads_com.iterrows():
    demand_estimate_com += demand_estimate(row[c['occ']])

col_area_com_sqft = demand_estimate_com * col_area_scaler
tank_vol_com_gal = col_area_com_sqft * tank_vol_scaler

col_area_com = UnitConv(col_area_com_sqft).sqft_m2()
tank_vol_com = UnitConv(tank_vol_com_gal).m3_gal(unit_in = 'gal')

* Gas tankless WH: Based on the WH efficiency standards analysis and input powers collected online

In [17]:
def gas_tankles_size_W(occupancy):
    size = 24875.*occupancy**0.5175
    return size

* Solar pump

In [18]:
# solar pump
solar_pump_w_indiv = 7.5101 * sum(occ_ind) ** 0.5322
solar_pump_w_com = 7.5101 * sum(occ_com) ** 0.5322

### Distribution system

* Piping

In [19]:
# If the length of community piping installation is unknown, use this estimate

# individual 
pipe_m_indiv = 0 # this can be set to a real value if it is known. It is set to 0 here since the part of piping inside of a 
# household is assumed to have negligible thermal losses

# community scale

pipe_m_per_hhld = 3.048
# household type distribution pipe length scalers for communities with attached and detached single family houses
multifamily = 1.
attached_k = 3.
detached_k = 6.

diameter_scaler=0.00791128
diameter_exponent=0.430827

# The average diameter of the distribution piping is a function of distribution pipe length and it gets autosized 
# within the distribution model, where the first larger value than that theoretically obtained gets selected
# from this list: [0.0127, 0.01905, 0.0254, 0.03175, 0.0381, 0.0508, 0.0635, 0.0762, 0.1016], in m

# The formula is:

# d_m = diameter_scaler * (pipe_m_per_hhld * number_of_households)**diameter_exponent
# To change these parameters, edit values in sol_the_sys_params_{com/indiv} dataframes

d_m = diameter_scaler * (pipe_m_per_hhld * len(occ_com))**diameter_exponent

d_m

0.02323577539362201

* Distribution pump

In [20]:
dist_pump_w = 10.4376 * len(occ_com) ** 0.9277

## Compile component sizes for system simulation

* Basecase gas tank WH

In [21]:
# Individual household

gas_tank_wh_size = pd.DataFrame(
    data = [[s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [s['comp'], s['cap'], s['cap_unit']])

gas_tank_wh_size

,Component,Component Size,Component Size Unit
0,gas tank WH,0.158987,m3


* Solar thermal system, individual household

In [22]:
# Main system

sys_sizes_indiv = pd.DataFrame(data = \
    [[s['sol_col'], col_area_ind, 'm2'], \
     [s['the_sto'], tank_vol_ind, 'm3'], \
     [s['sol_pump'], solar_pump_w_indiv, 'W'], \
     [s['piping'], pipe_m_indiv, 'm']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

sys_sizes_indiv

,Component,Component Size,Component Size Unit
0,solar collector,5.016764,m2
1,thermal storage tank,0.265736,m3
2,solar pump,13.476268,W
3,piping,0.000000,m


In [23]:
# Retrofit: Backup gas tank WH

retrofit_bckp_size_indiv = pd.DataFrame(data = \
            [[1, s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])
demand_estimate_com

180.0

In [24]:
# New: Backup gas tankless WH

new_bckp_size_ind = pd.DataFrame(data = [[1, s['gas_burn'], gas_tankles_size_W(occ_ind[0]), 'W']], \
            columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])

new_bckp_size_ind

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas burner,43921.113177,W


* Solar thermal system, community

In [25]:
# Main system components

sys_sizes_com = pd.DataFrame(data = \
            [[s['sol_col'], col_area_com, 'm2'], \
             [s['the_sto'], tank_vol_com, 'm3'], \
             [s['sol_pump'], solar_pump_w_com, 'W'], \
             [s['dist_pump'], dist_pump_w, 'W'], \
             [s['piping'], multifamily * pipe_m_per_hhld * len(occ_com),'m']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

In [26]:
# Retrofit: Backup gas tank WHs

retrofit_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']],
                                      index = peakload_com.index)

for i in peakload_com.index:
    retrofit_bckp_size_com.loc[i, c['id']] = i
    retrofit_bckp_size_com.loc[i, s['comp']] = s['gas_tank']
    retrofit_bckp_size_com.loc[i, s['cap']] = peakload_com.loc[i, 'gas_tank_size_com'].round(3)

retrofit_bckp_size_com.index = range(retrofit_bckp_size_com.shape[0])
retrofit_bckp_size_com[s['cap_unit']] = 'm3'
retrofit_bckp_size_com

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas tank WH,0.151,m3
1,2,gas tank WH,0.159,m3
2,3,gas tank WH,0.159,m3
3,4,gas tank WH,0.169,m3


In [27]:
# New: Backup gas tankless WHs

new_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']], index = peakload_com.index)

for i in peakload_com.index:
    new_bckp_size_com.loc[i, c['id']] = i
    new_bckp_size_com.loc[i, s['comp']] = s['gas_burn']
    new_bckp_size_com.loc[i, s['cap']] = gas_tankles_size_W(occ_com[i - 1])

new_bckp_size_com.index = range(new_bckp_size_com.shape[0])

* Validation sizing

Sizes based on several observed rated systems from OG-300 with an 80 gal tank and gas tankless backup
(https://secure.solar-rating.org/Certification/Ratings/RatingsReport.aspx?device=6926&units=METRICS)


In [28]:
# Solar thermal with tankless backup
sys_sizes_val = pd.DataFrame(data = \
            [[s['sol_col'], 3.9], \
             [s['the_sto'], UnitConv(80.).m3_gal(unit_in ='gal')], \
             [s['sol_pump'], solar_pump_w_indiv], \
             [s['piping'], 0.]], \
            columns = \
             [s['comp'], s['cap']])

sys_sizes_val

,Component,Component Size
0,solar collector,3.900000
1,thermal storage tank,0.302833
2,solar pump,13.476268
3,piping,0.000000


In [29]:
# visualisation setup
pd.options.display.float_format = '{:,.3f}'.format
w = 650
h = 450
f = 16
m_l = 60.
m_b = 60.

# hours of the year plotted time, a week in July starting on Monday
first_hour = 29*7*24 + 1
last_hour = first_hour + 7*24
first_hour

4873

# System Simulation
## Basecase gas tank WH

In [30]:
# cold climate

conv_gas_tank_wh_cold = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_cold, \
            loads = loads_indiv)

cons_total_cold, proj_total_cold, ts_proj_cold = conv_gas_tank_wh_cold.conventional_gas_tank()

# hot climate

conv_gas_tank_wh_hot = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_hot, \
            loads = loads_indiv)

cons_total_hot, proj_total_hot, ts_proj_hot = conv_gas_tank_wh_hot.conventional_gas_tank()

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [31]:
proj_total_cold/1000

Consumer ID                                    0.001
Occupancy                                      0.003
Energy Use - Gas                           4,387.181
Energy Use - Gas, Summer                   1,802.332
Energy Use - Gas, Winter                   2,584.849
Energy Use - Electricity                       0.000
Energy Use - Electricity, Summer               0.000
Energy Use - Electricity, Winter               0.000
Solar Fraction                                 0.000
Backup Heat Delivered                            NaN
Unmet Heat                                     0.000
Energy Use - Gas, w/o Dist Loss            4,387.181
Energy Use - Gas, Summer, w/o Dist Loss    1,802.332
Energy Use - Gas, Winter, w/o Dist Loss    2,584.849
Total Heat Delivered                       2,425.223
Net Heat Demand                            2,425.223
Dumped Heat                                    0.000
Distribution Heat Loss                         0.000
Distribution Heat Loss at Backup              

In [32]:
proj_total_hot/1000

Consumer ID                                    0.001
Occupancy                                      0.003
Energy Use - Gas                           3,516.334
Energy Use - Gas, Summer                   1,386.999
Energy Use - Gas, Winter                   2,129.335
Energy Use - Electricity                       0.000
Energy Use - Electricity, Summer               0.000
Energy Use - Electricity, Winter               0.000
Solar Fraction                                 0.000
Backup Heat Delivered                            NaN
Unmet Heat                                     0.000
Energy Use - Gas, w/o Dist Loss            3,516.334
Energy Use - Gas, Summer, w/o Dist Loss    1,386.999
Energy Use - Gas, Winter, w/o Dist Loss    2,129.335
Total Heat Delivered                       1,737.257
Net Heat Demand                            1,737.257
Dumped Heat                                    0.000
Distribution Heat Loss                         0.000
Distribution Heat Loss at Backup              

In [33]:
data_bc = pd.DataFrame(data = \
        [ts_proj_cold[r['gas_use']], \
         ts_proj_hot[r['gas_use']]]).transpose()

fig = Plot(data_headers = ['Gas Use, Cold Climate (Zone '+str(int(cold_cz))+')','Gas Use, Hot Climate (Zone '+hot_cz+')'], \
                 outpath = '', save_image = False, title = '', label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.01, legend_y = 1.).\
            series(data_bc[first_hour: last_hour], outfile = '', index_in_a_column = None, modes = 'lines')

## Individual new solar thermal system

In [34]:
# in a cold climate
sol_wh_indiv_new_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = new_bckp_size_ind,
                          weather = weather_cold,
                          loads = loads_indiv)

cons_total_indiv_new_cold, proj_total_indiv_new_cold,\
[proj_total_dict_indiv_new_cold, sol_fra_indiv_new_cold, pump_el_use_indiv_new_cold,\
 pump_op_hour_indiv_new_cold, ts_res_indiv_new_cold, backup_ts_cons_indiv_new_cold, rel_err_indiv_new_cold] = \
    sol_wh_indiv_new_cold.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.


In [35]:
print('month, day, hour:\nData starts at', ts_res_indiv_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Individual SWH New' + ', Climate Zone ' + str(int(cold_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.4).\
            series(ts_res_indiv_new_cold.loc[first_hour : last_hour,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [36]:
ts_res_indiv_new_cold.keys()

Index(['Net Heat Demand', 'Heat Demand With Dist. Loss',
       'Demand Balancing Error (Heat Rate)', 'Solar Heat Delivered To Tank',
       'Heat Loss - Lower Tank Volume', 'Heat Loss - Upper Tank Volume',
       'Tank Heat Delivered', 'Tank Unmet Heat', 'Dumped Heat',
       'Tank Overcool (Heat Rate)', 'Temperature - Upper Tank Volume',
       'Temperature - Lower Tank Volume', 'Temperature - Tank Coil Out',
       'Temperature - Hot Water Set', 'Temperature Drop in Distribution Pipes',
       'Distribution Heat Loss', 'Distribution Heat Loss at Backup',
       'Distribution Heat Loss at Backup, Summer',
       'Distribution Heat Loss at Backup, Winter', 'Backup Heat Delivered',
       'Energy Use - Gas', 'Energy Use - Gas, Summer',
       'Energy Use - Gas, Winter', 'Energy Use - Gas, w/o Dist Loss',
       'Energy Use - Gas, Summer, w/o Dist Loss',
       'Energy Use - Gas, Winter, w/o Dist Loss', 'Unmet Heat',
       'Total Heat Delivered', 'Temperature - Ambient',
       'Temper

In [37]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Individual SWH New, Climate Zone ' + str(int(cold_cz)), 
                 label_h = 'Hour [h]', label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.01, legend_y = 1.15).\
            series(ts_res_indiv_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

In [38]:
# in a hot climate
sol_wh_indiv_new_hot = System(sys_params = sol_the_sys_params_indiv, \
                          backup_params = gas_tankless_wh_params, \
                          sys_sizes = sys_sizes_indiv, \
                          backup_sizes = new_bckp_size_ind, \
                          weather = weather_hot, \
                          loads = loads_indiv)

cons_total_indiv_new_hot, proj_total_indiv_new_hot,\
[proj_total_dict_indiv_new_hot, sol_fra_indiv_new_hot, pump_el_use_indiv_new_hot,\
 pump_op_hour_indiv_new_hot, ts_res_indiv_new_hot, backup_ts_cons_indiv_new_hot, rel_err_indiv_new_hot] = \
    sol_wh_indiv_new_hot.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.


In [39]:
# first_hour = 4897
# last_hour = 4897 + 5*24

print('month, day, hour:\nData starts at', ts_res_indiv_new_hot.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Individual SWH New, Climate Zone ' + str(int(hot_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.02, legend_y = 1.4).\
            series(ts_res_indiv_new_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [40]:
# title = 'Individual SWH New, Climate Zone ' + str(int(hot_cz))

fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = '', label_v = 'Temperature [K]',\
                 label_h = 'Hour [h]', legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.3).\
            series(ts_res_indiv_new_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

### Compare individual new system annual totals for two climates

In [41]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_indiv_new_cold)
df = df.transpose()
df.index = ['Individual SWH, New, Cold Climate Zone ' + cold_cz]
df1 = pd.DataFrame(proj_total_indiv_new_hot)
df1 = df1.transpose()
df1.index = ['Individual SWH, New, Hot Climate Zone ' + hot_cz]
compare_indiv_new = df.append(df1).transpose()
compare_indiv_new/1000

,"Individual SWH, New, Cold Climate Zone 01","Individual SWH, New, Hot Climate Zone 15"
Net Heat Demand,"2,425.224","1,737.256"
Heat Demand With Dist. Loss,"2,425.224","1,737.256"
Demand Balancing Error (Heat Rate),0.000,0.000
Solar Heat Delivered To Tank,"2,840.269","4,589.625"
Heat Loss - Lower Tank Volume,187.498,198.007
Heat Loss - Upper Tank Volume,203.633,212.973
Tank Heat Delivered,"2,024.730","1,705.425"
Tank Unmet Heat,400.494,31.832
Dumped Heat,421.987,"2,465.112"
Tank Overcool (Heat Rate),0.009,0.000


In [42]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare_indiv_new.loc[["Net Heat Demand",
'Heat Demand With Dist. Loss',
'Solar Heat Delivered To Tank',
'Tank Heat Delivered',
'Tank Unmet Heat',
'Dumped Heat',
'Distribution Heat Loss',
'Backup Heat Delivered',
'Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Unmet Heat',
'Total Heat Delivered',
'Energy Use - Electricity',
'Energy Use - Electricity, Summer',
'Energy Use - Electricity, Winter',
'Solar Fraction',
'Peak End-Use Load [gal]'],:]/1000).round(0).to_clipboard()

## Community new solar thermal system

In [43]:
# in a cold climate
sol_wh_com_new_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = new_bckp_size_com,
                          weather = weather_cold,
                          loads = loads_com)

cons_total_com_new_cold, proj_total_com_new_cold,\
[proj_total_dict_com_new_cold, sol_fra_com_new_cold, pump_el_use_com_new_cold,\
 pump_op_hour_com_new_cold, ts_res_com_new_cold, backup_ts_cons_com_new_cold, rel_err_com_new_cold] = \
    sol_wh_com_new_cold.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.


In [44]:
# first_hour = 4897
# last_hour = 4897 + 5*24

print('month, day, hour:\nData starts at', ts_res_com_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Community SWH New, Climate Zone ' + str(int(cold_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.65, legend_y = 1.3).\
            series(ts_res_com_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [45]:
# in a hot climate
sol_wh_com_new_hot = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = new_bckp_size_com,
                          weather = weather_hot,
                          loads = loads_com)

cons_total_com_new_hot, proj_total_com_new_hot,\
[proj_total_dict_com_new_hot, sol_fra_com_new_hot, pump_el_use_com_new_hot,\
 pump_op_hour_com_new_hot, ts_res_com_new_hot, backup_ts_cons_com_new_hot, rel_err_com_new_hot] = \
    sol_wh_com_new_hot.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.


In [46]:
print('month, day, hour:\nData starts at', ts_res_com_new_hot.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_new_hot.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Community SWH New, Climate Zone ' + str(int(cold_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False,margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.65, legend_y = 1.5).\
            series(ts_res_com_new_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


### Compare community new system annual totals for two climates

In [47]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_com_new_cold)
df = df.transpose()
df.index = ['Community SWH, New, Cold Climate Zone ' + cold_cz]
df1 = pd.DataFrame(proj_total_com_new_hot)
df1 = df1.transpose()
df1.index = ['Community SWH, New, Hot Climate Zone ' + hot_cz]
compare_com_new = df.append(df1).transpose()
compare_com_new

,"Community SWH, New, Cold Climate Zone 01","Community SWH, New, Hot Climate Zone 15"
Net Heat Demand,"9,633,432","6,899,778"
Heat Demand With Dist. Loss,"9,708,428","6,976,292"
Demand Balancing Error (Heat Rate),0,0
Solar Heat Delivered To Tank,"10,993,628","18,159,028"
Heat Loss - Lower Tank Volume,"498,986","512,103"
Heat Loss - Upper Tank Volume,"528,581","541,248"
Tank Heat Delivered,"8,271,488","6,888,248"
Tank Unmet Heat,"1,436,940","88,043"
Dumped Heat,"1,684,028","10,184,794"
Tank Overcool (Heat Rate),24,0


In [48]:
# pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare_com_new.loc[["Net Heat Demand",
'Heat Demand With Dist. Loss',
'Solar Heat Delivered To Tank',
'Tank Heat Delivered',
'Tank Unmet Heat',
'Dumped Heat',
'Distribution Heat Loss',
'Backup Heat Delivered',
'Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Unmet Heat',
'Total Heat Delivered',
'Energy Use - Electricity',
'Energy Use - Electricity, Summer',
'Energy Use - Electricity, Winter',
'Solar Fraction',
'Peak End-Use Load [gal]'],:]/1000).round(0)#.to_clipboard()

,"Community SWH, New, Cold Climate Zone 01","Community SWH, New, Hot Climate Zone 15"
Net Heat Demand,"9,633","6,900"
Heat Demand With Dist. Loss,"9,708","6,976"
Solar Heat Delivered To Tank,"10,994","18,159"
Tank Heat Delivered,"8,271","6,888"
Tank Unmet Heat,"1,437",88
Dumped Heat,"1,684","10,185"
Distribution Heat Loss,75,77
Backup Heat Delivered,"1,437",88
Energy Use - Gas,"1,691",104
"Energy Use - Gas, Summer",231,0


In [49]:
cons_total_com_new_cold["Solar Fraction"].round(3)

0   1
1   1
2   1
3   1
Name: Solar Fraction, dtype: float64

In [50]:
# Project level total vs. individual household total
print(abs(cons_total_com_new_hot["Energy Use - Gas"].sum() -
                    proj_total_com_new_hot["Energy Use - Gas"]).round(2))

print(abs(cons_total_com_new_hot["Tank Heat Delivered"].sum() -
                    proj_total_com_new_hot["Tank Heat Delivered"]).round(2))

0.0
0.0


### Compare new installation systems across climates and scales

In [51]:
# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Per Household, Community, New, Cold Climate Zone ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual, New, Cold Climate Zone ' + cold_cz]
compare_cold = df.append(df1).transpose()

# for hot climate

# Energy use per household
com = cons_total_com_new_hot.sum()/4
ind = cons_total_indiv_new_hot.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Per Household, Community, New, Hot Climate Zone ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual, New, Hot Climate Zone ' + hot_cz]
compare_hot = df.append(df1).transpose()
compare = compare_cold.copy()
compare[compare_hot.columns] = compare_hot

compare

,"Per Household, Community, New, Cold Climate Zone 01","Individual, New, Cold Climate Zone 01","Per Household, Community, New, Hot Climate Zone 15","Individual, New, Hot Climate Zone 15"
Consumer ID,2,1,2,1
Occupancy,3,3,3,3
Energy Use - Gas,"422,629","471,170","25,895","37,449"
"Energy Use - Gas, Summer","57,774","65,169",11,232
"Energy Use - Gas, Winter","364,856","406,000","25,884","37,217"
Energy Use - Electricity,"29,169","45,423","33,364","52,621"
"Energy Use - Electricity, Summer","13,661","21,372","15,186","24,162"
"Energy Use - Electricity, Winter","15,509","24,051","18,178","28,459"
Solar Fraction,1,1,1,1
Backup Heat Delivered,"359,235","400,494","22,011","31,832"


In [52]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]/1000).round(0)#.to_clipboard()

,"Per Household, Community, New, Cold Climate Zone 01","Individual, New, Cold Climate Zone 01","Per Household, Community, New, Hot Climate Zone 15","Individual, New, Hot Climate Zone 15"
Energy Use - Gas,423,471,26,37
"Energy Use - Gas, Summer",58,65,0,0
"Energy Use - Gas, Winter",365,406,26,37
"Energy Use - Gas, w/o Dist Loss",417,471,25,37
Energy Use - Electricity,29,45,33,53
Solar Fraction,0,0,0,0
Backup Heat Delivered,359,400,22,32
Unmet Heat,0,0,0,0
Net Heat Demand,"2,408","2,425","1,725","1,737"
Total Heat Delivered,"2,427","2,425","1,744","1,737"


## Individual retrofit solar thermal system

In [53]:
# in a cold climate
sol_wh_indiv_retrofit_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = retrofit_bckp_size_indiv,
                          weather = weather_cold,
                          loads = loads_indiv)

cons_total_indiv_retrofit_cold, proj_total_indiv_retrofit_cold,\
[proj_total_dict_indiv_retrofit_cold, sol_fra_indiv_retrofit_cold, pump_el_use_indiv_retrofit_cold,\
 pump_op_hour_indiv_retrofit_cold, ts_res_indiv_retrofit_cold, backup_ts_cons_indiv_retrofit_cold, rel_err_indiv_retrofit_cold] = \
    sol_wh_indiv_retrofit_cold.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [54]:
print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', Climate Zone ' + str(int(cold_cz)),
                 label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [55]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', Climate Zone ' + str(int(cold_cz)), label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

In [56]:
# in a hot climate
sol_wh_indiv_retrofit_hot = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = retrofit_bckp_size_indiv,
                          weather = weather_hot,
                          loads = loads_indiv)

cons_total_indiv_retrofit_hot, proj_total_indiv_retrofit_hot,\
[proj_total_dict_indiv_retrofit_hot, sol_fra_indiv_retrofit_hot, pump_el_use_indiv_retrofit_hot,\
 pump_op_hour_indiv_retrofit_hot, ts_res_indiv_retrofit_hot, backup_ts_cons_indiv_retrofit_hot, rel_err_indiv_retrofit_hot] = \
    sol_wh_indiv_retrofit_hot.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [57]:
print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_hot.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_hot.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', Climate Zone ' + str(int(hot_cz)), label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [58]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', Climate Zone ' + str(int(hot_cz)), label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

### Compare individual retrofit system annual totals for two climates

In [59]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_indiv_retrofit_cold)
df = df.transpose()
df.index = ['Individual SWH, Retrofit, Cold Climate Zone ' + cold_cz]
df1 = pd.DataFrame(proj_total_indiv_retrofit_hot)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, Cold Climate Zone ' + hot_cz]
compare_indiv_retrofit = df.append(df1).transpose()
compare_indiv_retrofit

,"Individual SWH, Retrofit, Cold Climate Zone 01","Individual SWH, Retrofit, Cold Climate Zone 15"
Net Heat Demand,"2,425,224","1,737,256"
Heat Demand With Dist. Loss,"2,425,224","1,737,256"
Demand Balancing Error (Heat Rate),0,0
Solar Heat Delivered To Tank,"2,840,269","4,589,625"
Heat Loss - Lower Tank Volume,"187,498","198,007"
Heat Loss - Upper Tank Volume,"203,633","212,973"
Tank Heat Delivered,"2,024,730","1,705,425"
Tank Unmet Heat,"400,494","31,832"
Dumped Heat,"421,987","2,465,112"
Tank Overcool (Heat Rate),9,0


In [60]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare_indiv_retrofit.loc[["Net Heat Demand",
'Heat Demand With Dist. Loss',
'Solar Heat Delivered To Tank',
'Tank Heat Delivered',
'Tank Unmet Heat',
'Dumped Heat',
'Distribution Heat Loss',
'Backup Heat Delivered',
'Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Unmet Heat',
'Total Heat Delivered',
'Energy Use - Electricity',
'Energy Use - Electricity, Summer',
'Energy Use - Electricity, Winter',
'Solar Fraction',
'Peak End-Use Load [gal]'],:]/1000).round(0).to_clipboard()

## Community retrofit solar thermal system

In [61]:
# in a cold climate
sol_wh_com_retrofit_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = retrofit_bckp_size_com,
                          weather = weather_cold,
                          loads = loads_com)

cons_total_com_retrofit_cold, proj_total_com_retrofit_cold,\
[proj_total_dict_com_retrofit_cold, sol_fra_com_retrofit_cold, pump_el_use_com_retrofit_cold,\
 pump_op_hour_com_retrofit_cold, ts_res_com_retrofit_cold, backup_ts_cons_com_retrofit_cold, rel_err_com_retrofit_cold] = \
    sol_wh_com_retrofit_cold.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [62]:
print('month, day, hour:\nData starts at', ts_res_com_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Community SWH Retrofit, Climate Zone ' + str(int(cold_cz)), label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.).\
            series(ts_res_com_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [63]:
# in a hot climate
sol_wh_com_retrofit_hot = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = retrofit_bckp_size_com,
                          weather = weather_hot,
                          loads = loads_com)

cons_total_com_retrofit_hot, proj_total_com_retrofit_hot,\
[proj_total_dict_com_retrofit_hot, sol_fra_com_retrofit_hot, pump_el_use_com_retrofit_hot,\
 pump_op_hour_com_retrofit_hot, ts_res_com_retrofit_hot, backup_ts_cons_com_retrofit_hot, rel_err_com_retrofit_hot] = \
    sol_wh_com_retrofit_hot.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [64]:
print('month, day, hour:\nData starts at', ts_res_com_retrofit_hot.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_hot.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Community SWH Retrofit, Climate Zone ' + str(int(hot_cz)), label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_com_retrofit_hot.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


### Compare community retrofit system annual totals for two climates

In [65]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_com_retrofit_cold)
df = df.transpose()
df.index = ['Community SWH, Retrofit, Cold Climate ' + cold_cz]
df1 = pd.DataFrame(proj_total_com_retrofit_hot)
df1 = df1.transpose()
df1.index = ['Community SWH, Retrofit, Hot Climate ' + hot_cz]
compare_community_retrofit = df.append(df1).transpose()
compare_community_retrofit.round(3)

,"Community SWH, Retrofit, Cold Climate 01","Community SWH, Retrofit, Hot Climate 15"
Net Heat Demand,"9,633,432","6,899,778"
Heat Demand With Dist. Loss,"9,708,428","6,976,292"
Demand Balancing Error (Heat Rate),0,0
Solar Heat Delivered To Tank,"10,993,628","18,159,028"
Heat Loss - Lower Tank Volume,"498,986","512,103"
Heat Loss - Upper Tank Volume,"528,581","541,248"
Tank Heat Delivered,"8,271,488","6,888,248"
Tank Unmet Heat,"1,436,940","88,043"
Dumped Heat,"1,684,028","10,184,794"
Tank Overcool (Heat Rate),24,0


In [66]:
cons_total_com_retrofit_cold["Solar Fraction"].round(3)

0   1
1   1
2   1
3   1
Name: Solar Fraction, dtype: float64

In [67]:
pd.options.display.float_format = '{:,.3f}'.format
# subset of results for the report
(compare_community_retrofit.loc[["Net Heat Demand",
'Heat Demand With Dist. Loss',
'Solar Heat Delivered To Tank',
'Tank Heat Delivered',
'Tank Unmet Heat',
'Dumped Heat',
'Distribution Heat Loss',
'Backup Heat Delivered',
'Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Unmet Heat',
'Total Heat Delivered',
'Energy Use - Electricity',
'Energy Use - Electricity, Summer',
'Energy Use - Electricity, Winter',
'Solar Fraction',
'Peak End-Use Load [gal]'],:]/1000).round(0)#.to_clipboard()

,"Community SWH, Retrofit, Cold Climate 01","Community SWH, Retrofit, Hot Climate 15"
Net Heat Demand,"9,633.000","6,900.000"
Heat Demand With Dist. Loss,"9,708.000","6,976.000"
Solar Heat Delivered To Tank,"10,994.000","18,159.000"
Tank Heat Delivered,"8,271.000","6,888.000"
Tank Unmet Heat,"1,437.000",88.000
Dumped Heat,"1,684.000","10,185.000"
Distribution Heat Loss,75.000,77.000
Backup Heat Delivered,"1,436.000",88.000
Energy Use - Gas,"7,098.000","5,391.000"
"Energy Use - Gas, Summer","2,461.000","2,213.000"


In [68]:
# Project level total vs. individual household total
print(abs(cons_total_com_retrofit_hot["Energy Use - Gas"].sum() -
                    proj_total_com_retrofit_hot["Energy Use - Gas"]).round(2))

print(abs(cons_total_com_retrofit_hot["Tank Heat Delivered"].sum() -
                    proj_total_com_retrofit_hot["Tank Heat Delivered"]).round(2))

0.0
0.0


## Compare systems across climates, configurations and scales

### Retrofits

In [69]:
# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, Cold Climate ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, Cold Climate ' + cold_cz]
compare_cold = df.append(df1).transpose()

'Community SWH, Retrofit, Cold Climate ' + cold_cz

# for hot climate

# Energy use per household
com = cons_total_com_retrofit_hot.sum()/4
ind = cons_total_indiv_retrofit_hot.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, Hot Climate ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, Hot Climate ' + hot_cz]
compare_hot = df.append(df1).transpose()
compare = compare_cold.copy()
compare[compare_hot.columns] = compare_hot

compare.drop(index='Consumer ID', axis = 1, inplace = True)
compare

,"Community SWH, Per Household, Retrofit, Cold Climate 01","Individual SWH, Retrofit, Cold Climate 01","Community SWH, Per Household, Retrofit, Hot Climate 15","Individual SWH, Retrofit, Hot Climate 15"
Occupancy,3.000,3.000,3.000,3.000
Energy Use - Gas,"1,774,481.312","1,824,376.783","1,347,836.479","1,357,709.200"
"Energy Use - Gas, Summer","615,349.084","622,287.786","553,269.763","552,418.111"
"Energy Use - Gas, Winter","1,159,132.228","1,202,088.996","794,566.716","805,291.088"
Energy Use - Electricity,"29,169.470","45,422.950","33,363.750","52,620.860"
"Energy Use - Electricity, Summer","13,660.580","21,371.780","15,185.770","24,162.160"
"Energy Use - Electricity, Winter","15,508.890","24,051.180","18,177.980","28,458.710"
Solar Fraction,0.850,0.830,0.990,0.980
Backup Heat Delivered,"359,025.980","400,497.466","21,980.587","31,831.540"
Unmet Heat,0.000,0.000,0.000,0.000


In [70]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]/1000).round(0)#.to_clipboard()

,"Community SWH, Per Household, Retrofit, Cold Climate 01","Individual SWH, Retrofit, Cold Climate 01","Community SWH, Per Household, Retrofit, Hot Climate 15","Individual SWH, Retrofit, Hot Climate 15"
Energy Use - Gas,"1,774","1,824","1,348","1,358"
"Energy Use - Gas, Summer",615,622,553,552
"Energy Use - Gas, Winter","1,159","1,202",795,805
"Energy Use - Gas, w/o Dist Loss","1,769","1,824","1,347","1,358"
Energy Use - Electricity,29,45,33,53
Solar Fraction,0,0,0,0
Backup Heat Delivered,359,400,22,32
Unmet Heat,0,0,0,0
Net Heat Demand,"2,408","2,425","1,725","1,737"
Total Heat Delivered,"2,427","2,425","1,744","1,737"


### New and Retrofit

In [71]:
# New installations

# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, New, Cold Climate ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, Cold Climate ' + cold_cz]
compare_cold = df.append(df1).transpose()
compare = compare_cold.copy()

# for hot climate

# Energy use per household
com = cons_total_com_new_hot.sum()/4
ind = cons_total_indiv_new_hot.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, New, Hot Climate ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, New, Hot Climate ' + hot_cz]
compare_hot = df.append(df1).transpose()
compare[compare_hot.columns] = compare_hot


# Retrofits

# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, Cold Climate ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, cold CZ' + cold_cz]
compare_cold_retr = df.append(df1).transpose()
compare[compare_cold_retr.columns] = compare_cold_retr

# for hot climate

# Energy use per household
com = cons_total_com_retrofit_hot.sum()/4
ind = cons_total_indiv_retrofit_hot.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, Hot Climate ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, Hot Climate ' + hot_cz]
compare_hot_retr = df.append(df1).transpose()
compare[compare_hot_retr.columns] = compare_hot_retr

subset = compare.drop(['Consumer ID'], axis = 0).loc[["Net Heat Demand", "Total Heat Delivered",
        "Energy Use - Gas","Energy Use - Gas, w/o Dist Loss",
        "Energy Use - Electricity", "Solar Fraction", "Backup Heat Delivered",
        "Unmet Heat", "Distribution Heat Loss"],:]
compare

,"Community SWH, Per Household, New, Cold Climate 01","Individual SWH, Retrofit, Cold Climate 01","Community SWH, Per Household, New, Hot Climate 15","Individual SWH, New, Hot Climate 15","Community SWH, Per Household, Retrofit, Cold Climate 01","indiv, retrofit, cold CZ01","Community SWH, Per Household, Retrofit, Hot Climate 15","Individual SWH, Retrofit, Hot Climate 15"
Consumer ID,2,1,2,1,2,1,2,1
Occupancy,3,3,3,3,3,3,3,3
Energy Use - Gas,"422,629","471,170","25,895","37,449","1,774,481","1,824,377","1,347,836","1,357,709"
"Energy Use - Gas, Summer","57,774","65,169",11,232,"615,349","622,288","553,270","552,418"
"Energy Use - Gas, Winter","364,856","406,000","25,884","37,217","1,159,132","1,202,089","794,567","805,291"
Energy Use - Electricity,"29,169","45,423","33,364","52,621","29,169","45,423","33,364","52,621"
"Energy Use - Electricity, Summer","13,661","21,372","15,186","24,162","13,661","21,372","15,186","24,162"
"Energy Use - Electricity, Winter","15,509","24,051","18,178","28,459","15,509","24,051","18,178","28,459"
Solar Fraction,1,1,1,1,1,1,1,1
Backup Heat Delivered,"359,235","400,494","22,011","31,832","359,026","400,497","21,981","31,832"


In [72]:
subset

,"Community SWH, Per Household, New, Cold Climate 01","Individual SWH, Retrofit, Cold Climate 01","Community SWH, Per Household, New, Hot Climate 15","Individual SWH, New, Hot Climate 15","Community SWH, Per Household, Retrofit, Cold Climate 01","indiv, retrofit, cold CZ01","Community SWH, Per Household, Retrofit, Hot Climate 15","Individual SWH, Retrofit, Hot Climate 15"
Net Heat Demand,"2,408,358","2,425,224","1,724,944","1,737,256","2,408,358","2,425,224","1,724,944","1,737,256"
Total Heat Delivered,"2,427,107","2,425,224","1,744,073","1,737,256","2,426,898","2,425,227","1,744,043","1,737,256"
Energy Use - Gas,"422,629","471,170","25,895","37,449","1,774,481","1,824,377","1,347,836","1,357,709"
"Energy Use - Gas, w/o Dist Loss","416,849","471,170","25,247","37,449","1,768,528","1,824,377","1,347,177","1,357,709"
Energy Use - Electricity,"29,169","45,423","33,364","52,621","29,169","45,423","33,364","52,621"
Solar Fraction,1,1,1,1,1,1,1,1
Backup Heat Delivered,"359,235","400,494","22,011","31,832","359,026","400,497","21,981","31,832"
Unmet Heat,0,0,0,0,0,0,0,0
Distribution Heat Loss,"18,749",0,"19,129",0,"18,749",0,"19,129",0


In [73]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]).round(0)#.to_clipboard()

,"Community SWH, Per Household, New, Cold Climate 01","Individual SWH, Retrofit, Cold Climate 01","Community SWH, Per Household, New, Hot Climate 15","Individual SWH, New, Hot Climate 15","Community SWH, Per Household, Retrofit, Cold Climate 01","indiv, retrofit, cold CZ01","Community SWH, Per Household, Retrofit, Hot Climate 15","Individual SWH, Retrofit, Hot Climate 15"
Energy Use - Gas,"422,629","471,170","25,895","37,449","1,774,481","1,824,377","1,347,836","1,357,709"
"Energy Use - Gas, Summer","57,774","65,169",11,232,"615,349","622,288","553,270","552,418"
"Energy Use - Gas, Winter","364,856","406,000","25,884","37,217","1,159,132","1,202,089","794,567","805,291"
"Energy Use - Gas, w/o Dist Loss","416,849","471,170","25,247","37,449","1,768,528","1,824,377","1,347,177","1,357,709"
Energy Use - Electricity,"29,169","45,423","33,364","52,621","29,169","45,423","33,364","52,621"
Solar Fraction,1,1,1,1,1,1,1,1
Backup Heat Delivered,"359,235","400,494","22,011","31,832","359,026","400,497","21,981","31,832"
Unmet Heat,0,0,0,0,0,0,0,0
Net Heat Demand,"2,408,358","2,425,224","1,724,944","1,737,256","2,408,358","2,425,224","1,724,944","1,737,256"
Total Heat Delivered,"2,427,107","2,425,224","1,744,073","1,737,256","2,426,898","2,425,227","1,744,043","1,737,256"


### Duration curves for individual households gas use

In [74]:
# climates: hot and cold
# configrations: retrofit and new

# single community system household

print('month, day, hour:\nData starts at', ts_res_com_retrofit_hot.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_hot.loc[last_hour, ['month','day', 'hour']].transpose().values)

df_gas_use = pd.DataFrame()

df_gas_use["New SWH, Cold Climate, Household 1"] = np.array(backup_ts_cons_com_new_cold[1][r['gas_use']])
df_gas_use["New SWH, Hot Climate, Household 1"] = np.array(backup_ts_cons_com_new_hot[1][r['gas_use']])
df_gas_use["Retrofit SWH, Cold Climate, Household 1"] = np.array(backup_ts_cons_com_retrofit_cold[1][r['gas_use']])
df_gas_use["Retrofit SWH, Hot Climate, Household 1"] = np.array(backup_ts_cons_com_retrofit_hot[1][r['gas_use']])

fig = Plot(data_headers = None, \
                 outpath = '', save_image = False, title = '', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = True, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = .01, legend_y = 1.2).\
            series(df_gas_use, outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


## Solar thermal system validation

We performed validation with average energy savings and solar fractions for climate zone 3 for solar thermal systems with an instantaneous backup provided in the [SRCC OG-300](https://secure.solar-rating.org/Certification/Ratings/RatingsSummaryPage.aspx?type=2) rating data.

We also performed the component models validation. Those validation tests can be found in `swh.system.tests.test_components`.

In [75]:
# scaled loads to match 300L/day, as used to obtain the rating data
load_array_val = loads_indiv['End-Use Load'][0] * \
    (300*365*.001)/loads_indiv['End-Use Load'][0].sum()

loads_indiv_val = pd.DataFrame(data = \
     [[1, occ_ind[0], load_array_val]],\
    columns = \
     [c['id'], c['occ'], c['load_m3']])

In [76]:
# in a cold climate
sol_wh_indiv_new_val = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_val,
                          backup_sizes = new_bckp_size_ind,
                          weather = weather_val,
                          loads = loads_indiv_val)

cons_total_indiv_new_val, proj_total_indiv_new_val,\
[proj_total_dict_indiv_new_val, sol_fra_indiv_new_val, pump_el_use_indiv_new_val,\
 pump_op_hour_indiv_new_val, ts_res_indiv_new_val, backup_ts_cons_indiv_new_val, rel_err_indiv_new_val] = \
    sol_wh_indiv_new_val.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.


In [77]:
# compare solar fraction for CZ03
rating_sheet_03 = 0.69
average_from_rating_sheet_for_similar_installations_03 = 0.72

proj_total_indiv_new_val['Solar Fraction']

0.68

In [78]:
# compare savings to basecase with the parameters corresponding the DOE appliance efficiency level 1
basecase_wh_val = System(sys_params = gas_tank_wh_params, 
            sys_sizes = gas_tank_wh_size, 
            weather = weather_val, 
            loads = loads_indiv_val)
bc_cons_total_el, bc_proj_total_el, bc_ts_proj_el = \
            basecase_wh_val.conventional_gas_tank()

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [79]:
# compare savings (note that there is no information in the rating sheet on the basecase tank WH)

rating_sheet_03 = 4490
average_from_rating_sheet_for_similar_installations_03 = 4707.18

abs(((bc_proj_total_el['Energy Use - Gas']
    - proj_total_indiv_new_val['Energy Use - Gas'])/1000
    - average_from_rating_sheet_for_similar_installations_03)/average_from_rating_sheet_for_similar_installations_03).round(2)

0.14

In [80]:
# With baseline tank with a slightly higher RE and insulation set to R12, as used for the solar tank

gas_tank_wh_params_val = pd.DataFrame(data = \
    [[s['gas_tank'], s['tank_re'], .82, '-'], 
     [s['gas_tank'], s['ins_thi'], .04, 'm'], 
     [s['gas_tank'], s['spec_hea_con'], .085, 'W/mK'], 
     [s['gas_tank'], s['t_tap_set'], 322.04, 'K']], 
    columns = \
     [s['comp'], s['param'], s['param_value'], s['param_unit']])
basecase_wh_val = System(sys_params = gas_tank_wh_params_val,
            sys_sizes = gas_tank_wh_size, 
            weather = weather_val,
            loads = loads_indiv_val)
bc_cons_total, bc_proj_total, bc_ts_proj = \
            basecase_wh_val.conventional_gas_tank()

# gas use savings compared to baseline
(bc_proj_total['Energy Use - Gas'] - proj_total_indiv_new_val['Energy Use - Gas'])/1000

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


4839.343414647049